In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl
import time
import dask.dataframe as dd
from fastparquet import write
import fastparquet
import multiprocessing as mp
import snappy
import argparse

In [2]:
parser = argparse.ArgumentParser(description='Program to combine pickle data into parquet files for version 3')

parser.add_argument('--server', type=str,
                    help='which server to be used')
parser.add_argument('--spath', type=str,
                    help='directory where to save output')

args = parser.parse_args('--server drogium --spath /data/dharp/compounds/datasets/V3/'.split())

pkl_df=pd.read_csv(f'/data/dharp/compounds/datasets/{args.server}_fcat.txt',sep="\t")



In [3]:
save_path=args.spath

In [5]:
cur_df=pkl_df.loc[pkl_df.fcat==90].copy()
cur_df

,fname,fcat
1796,/datanaco/dharp/compounds/datasets/googleV3/16...,90
1797,/datanaco/dharp/compounds/datasets/googleV3/24...,90
1798,/datanaco/dharp/compounds/datasets/googleV3/16...,90
1799,/datanaco/dharp/compounds/datasets/googleV3/16...,90
1800,/datanaco/dharp/compounds/datasets/googleV3/13...,90
...,...,...
1969,/datanaco/dharp/compounds/datasets/googleV3/26...,90
1970,/datanaco/dharp/compounds/datasets/googleV3/17...,90
1971,/datanaco/dharp/compounds/datasets/googleV3/17...,90
1972,/datanaco/dharp/compounds/datasets/googleV3/18...,90


In [6]:
    num_partitions=round(0.95*mp.cpu_count())
    print(cur_df.iloc[0].fcat)
    #cur_time=time.time()
    df_list=[]
    print(cur_df.shape[0])
    for row in cur_df.itertuples():
        #print(row.fname)
        cur_df=pd.read_pickle(row.fname)
        #cur_df.reset_index(inplace=True)
        df_list.append(cur_df)
    concat_df=pd.concat(df_list,ignore_index=True,sort=False)
    print(concat_df.shape[0])
    total_df_shape=concat_df.shape[0]
    print('Done concatenating')

90
178
1104296345
Done concatenating


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104296345 entries, 0 to 1104296344
Data columns (total 6 columns):
 #   Column         Dtype   
---  ------         -----   
 0   lemma_pos      category
 1   year           int64   
 2   comp_class     category
 3   num_comp       bool    
 4   comp_ner_sent  category
 5   count          int64   
dtypes: bool(1), category(3), int64(2)
memory usage: 25.9 GB


In [13]:
ddf = dd.from_pandas(concat_df, npartitions=100)

KeyboardInterrupt: 

In [18]:
943405885/1054019966

0.895055042059801

In [27]:
concat_df

lemma_pos                                                   year  comp_class  num_comp  comp_ner_sent
beat_NOUN anything_PRON i_PRON ever_ADV see_VERB            1889  0           False                       2
                                                            1894  0           False                       1
                                                            1896  0           False                       1
                                                            1899  0           False                       1
                                                            1901  0           False                       1
                                                                                                         ..
v'ıctor_X padrón_PROPN ,_PUNCT classification_PROPN of_ADP  2007  0           True                        1
                                                            2008  0           True                        3
                                  